In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [11]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain_ollama import ChatOllama

model = ChatOllama(model="lfm2.5-thinking")

class EmailState(AgentState):
    email: str

agent = create_agent(
    model=model,
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": False,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [12]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response immediately. Send the reply now in the same thread.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [13]:
from pprint import pprint

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now in the same thread.', additional_kwargs={}, response_metadata={}, id='9114ed11-260a-427b-ba87-ccea68fd767f'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-20T09:35:22.963832Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6101497666, 'load_duration': 33250125, 'prompt_eval_count': 132, 'prompt_eval_duration': 43301083, 'eval_count': 798, 'eval_duration': 5923659767, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c7a67-4f7c-7de1-8f80-e6e9b13d1879-0', tool_calls=[{'name': 'send_email', 'args': {'body': 'Your response to my email is: I will address your inquiry promptly.'}, 'id': 'a557f134-19db-4e03-aa1e-32fb8b6350a1', 'type': 'tool

In [9]:
print(response['__interrupt__'])

KeyError: '__interrupt__'

In [10]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

KeyError: '__interrupt__'

## Approve

In [8]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now in the same thread.', additional_kwargs={}, response_metadata={}, id='9aa16655-03d6-4d66-afa5-2dbd50f4824f'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-20T09:33:53.642741Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6655710542, 'load_duration': 36691667, 'prompt_eval_count': 132, 'prompt_eval_duration': 134236625, 'eval_count': 852, 'eval_duration': 6376811987, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c7a65-f069-7c71-af93-4dd6a1d47d98-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 132, 'output_tokens': 852, 'total_tokens': 984})]}


## Reject

In [ ]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Seán."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

In [ ]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

## Edit

In [14]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now in the same thread.', additional_kwargs={}, response_metadata={}, id='9114ed11-260a-427b-ba87-ccea68fd767f'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-20T09:35:22.963832Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6101497666, 'load_duration': 33250125, 'prompt_eval_count': 132, 'prompt_eval_duration': 43301083, 'eval_count': 798, 'eval_duration': 5923659767, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c7a67-4f7c-7de1-8f80-e6e9b13d1879-0', tool_calls=[{'name': 'send_email', 'args': {'body': 'Your response to my email is: I will address your inquiry promptly.'}, 'id': 'a557f134-19db-4e03-aa1e-32fb8b6350a1', 'type': 'tool